# <div style="text-align: center">AVALIANDO CACHORROS</div>

<div style="text-align: right">Projeto deselvolvido por Nadson Brito Gondim</div>
<div style="text-align: right">para fins de aprovação no nanodegree</div>
<div style="text-align: right">"Fundamentos de Data Science II" da Udacity</div>

## __INTRODUÇÃO__

## __PREPARAÇÃO__

## Coletando os dados

In [1]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import time
import math

In [2]:
#Fazendo o download do arquivo 'image-predictions.tsv'
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open(url.split('/')[-1], mode='wb') as file:
    file.write(response.content)

In [3]:
#Fazendo a leitura dos arquivos 'twitter-archive-enhanced.csv' e 'image-predictions.tsv'
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
image_predictions = pd.read_csv('image-predictions.tsv', sep="\s", engine='python')

In [4]:
#Acessando a API Tweepy
CONSUMER_KEY = 'XXXXXXXXXXX'
CONSUMER_SECRET = 'XXXXXXXXXXXXXXXXXXXXXxx'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
api = tweepy.API(auth)

In [5]:
#Armazenando os dados 'json' de cada tweet em um arquivo de texto 'tweet_json.txt'
file = open('tweet_json.txt', 'w', encoding='utf-8')
tweet_errors = {}
for idx, tweet_id in twitter_archive['tweet_id'].iteritems():
    attempt = False
    while attempt == False:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            tweet_json = str(tweet._json)+'\n'
            file.write(tweet_json)
            attempt = True
        except Exception as e:
            print(str(idx) + "_" + str(tweet_id) + ": " + str(e))
            if str(e) == "[{'message': 'Rate limit exceeded', 'code': 88}]":
                time.sleep(60 * 15)
                attempt = False
            else:
                tweet_errors[str(idx) + "_" + str(tweet_id)] = str(e)
                attempt = True
file.close()

19_888202515573088257: [{'code': 144, 'message': 'No status found with that ID.'}]
95_873697596434513921: [{'code': 144, 'message': 'No status found with that ID.'}]
118_869988702071779329: [{'code': 144, 'message': 'No status found with that ID.'}]
132_866816280283807744: [{'code': 144, 'message': 'No status found with that ID.'}]
155_861769973181624320: [{'code': 144, 'message': 'No status found with that ID.'}]
247_845459076796616705: [{'code': 144, 'message': 'No status found with that ID.'}]
260_842892208864923648: [{'code': 144, 'message': 'No status found with that ID.'}]
298_837012587749474308: [{'code': 144, 'message': 'No status found with that ID.'}]
382_827228250799742977: [{'code': 144, 'message': 'No status found with that ID.'}]
566_802247111496568832: [{'code': 144, 'message': 'No status found with that ID.'}]
784_775096608509886464: [{'code': 144, 'message': 'No status found with that ID.'}]
900_758740312047005698: [{'message': 'Rate limit exceeded', 'code': 88}]
932_7

In [6]:
# Lista de dicionários para construir linha por linha do arquivo '.txt' e depois converter em um dataframe
tweepy_list = []
with open('tweet_json.txt', 'r', encoding='utf-8') as file: 
    for line in file:
        json_acceptable_string = line[:-1].replace("\"", "'").replace("{'", "{\"").replace("':", "\":").replace(
                    "\": '", "\": \"").replace("',", "\",").replace(", '", ", \"").replace("'}", "\"}").replace(
                    "True", "true").replace("False", "false").replace("None", "null").replace("\\", "\\\\")
        
        try:
            json_line = json.loads(json_acceptable_string)
            tweet_id = json_line['id_str']
            retweet_count = json_line['retweet_count']
            favorite_count = json_line['favorite_count']
            tweepy_list.append({'tweet_id': tweet_id, 'retweet_count': retweet_count, 'favorite_count': favorite_count})
        except Exception as e:
            print(json_acceptable_string + "\n" + str(e) + "\n\n")
df_tweepy = pd.DataFrame(tweepy_list, columns = ['tweet_id', 'retweet_count', 'favorite_count'])
file.close()

## Avaliando os dados

In [7]:
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [8]:
image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [9]:
df_tweepy.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8558,38677
1,892177421306343426,6291,33157
2,891815181378084864,4171,24959
3,891689557279858688,8687,42061
4,891327558926688256,9447,40214


In [10]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [11]:

image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [12]:
df_tweepy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2344 entries, 0 to 2343
Data columns (total 3 columns):
tweet_id          2344 non-null object
retweet_count     2344 non-null int64
favorite_count    2344 non-null int64
dtypes: int64(2), object(1)
memory usage: 55.0+ KB


In [13]:
twitter_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [14]:
image_predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [15]:
df_tweepy.describe()

,retweet_count,favorite_count
count,2344.000000,2344.000000
mean,3013.664249,8042.073805
std,5014.418266,12104.841519
min,0.000000,0.000000
25%,604.750000,1400.000000
50%,1403.500000,3524.500000
75%,3506.500000,9943.250000
max,77112.000000,142959.000000


In [16]:
twitter_archive['name'].value_counts()

None         745
a             55
Charlie       12
Oliver        11
Lucy          11
Cooper        11
Tucker        10
Penny         10
Lola          10
Bo             9
Winston        9
Sadie          8
the            8
Bailey         7
an             7
Buddy          7
Toby           7
Daisy          7
Stanley        6
Dave           6
Jax            6
Bella          6
Jack           6
Koda           6
Milo           6
Oscar          6
Rusty          6
Scout          6
Leo            6
Bentley        5
            ... 
Divine         1
Lulu           1
Billy          1
Halo           1
Burt           1
Bronte         1
Mutt           1
Poppy          1
Traviss        1
Cermet         1
Robin          1
Schnozz        1
Stark          1
Pluto          1
Ozzie          1
Scruffers      1
Cora           1
Lassie         1
Kona           1
Tedders        1
Jeremy         1
Antony         1
Henry          1
Maks           1
Holly          1
Deacon         1
Snickers       1
Sundance      

In [17]:
twitter_archive['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [18]:
twitter_archive['doggo'].value_counts()

None     2259
doggo      97
Name: doggo, dtype: int64

In [19]:
twitter_archive['floofer'].value_counts()

None       2346
floofer      10
Name: floofer, dtype: int64

In [20]:
twitter_archive['pupper'].value_counts()

None      2099
pupper     257
Name: pupper, dtype: int64

In [21]:
twitter_archive['puppo'].value_counts()

None     2326
puppo      30
Name: puppo, dtype: int64

In [22]:
image_predictions['p1'].value_counts()

golden_retriever             150
Labrador_retriever           100
Pembroke                      89
Chihuahua                     83
pug                           57
chow                          44
Samoyed                       43
toy_poodle                    39
Pomeranian                    38
cocker_spaniel                30
malamute                      30
French_bulldog                26
Chesapeake_Bay_retriever      23
miniature_pinscher            23
seat_belt                     22
German_shepherd               20
Siberian_husky                20
Staffordshire_bullterrier     20
web_site                      19
Cardigan                      19
Maltese_dog                   18
teddy                         18
Shetland_sheepdog             18
beagle                        18
Eskimo_dog                    18
Shih-Tzu                      17
Rottweiler                    17
Lakeland_terrier              17
Italian_greyhound             16
kuvasz                        16
          

In [23]:
image_predictions['p2'].value_counts()

Labrador_retriever                104
golden_retriever                   92
Cardigan                           73
Chihuahua                          44
Pomeranian                         42
Chesapeake_Bay_retriever           41
French_bulldog                     41
toy_poodle                         37
cocker_spaniel                     34
miniature_poodle                   33
Siberian_husky                     33
beagle                             28
Eskimo_dog                         27
collie                             27
Pembroke                           27
kuvasz                             26
Italian_greyhound                  22
American_Staffordshire_terrier     21
Pekinese                           21
miniature_pinscher                 20
chow                               20
Samoyed                            20
toy_terrier                        20
malinois                           20
Norwegian_elkhound                 19
Boston_bull                        19
Staffordshir

In [24]:
image_predictions['p3'].value_counts()

Labrador_retriever                79
Chihuahua                         58
golden_retriever                  48
Eskimo_dog                        38
kelpie                            35
kuvasz                            34
chow                              32
Staffordshire_bullterrier         32
cocker_spaniel                    31
beagle                            31
Pekinese                          29
Pomeranian                        29
toy_poodle                        29
Great_Pyrenees                    27
Chesapeake_Bay_retriever          27
Pembroke                          27
malamute                          26
French_bulldog                    26
American_Staffordshire_terrier    24
Cardigan                          23
pug                               23
basenji                           21
bull_mastiff                      20
toy_terrier                       20
Siberian_husky                    19
Shetland_sheepdog                 17
Boston_bull                       17
b

In [25]:
sum(twitter_archive['tweet_id'].duplicated())

0

In [26]:
sum(image_predictions['tweet_id'].duplicated())

0

In [27]:
sum(df_tweepy['tweet_id'].duplicated())

0

In [28]:
sum(twitter_archive['tweet_id'].isnull())

0

In [29]:
sum(image_predictions['tweet_id'].isnull())

0

In [30]:
sum(df_tweepy['tweet_id'].isnull())

0

### Problemas de qualidade:

##### - tabela `twitter_archive`
- As variáveis 'timestamp' e 'retweeted_status_timestamp' são com tipo objeto (string), mas deveriam ser do tipo 'datatime';
- As variáveis que representam IDs são todas do tipo 'int' ou 'float', mas deveriam ser do tipo 'string';
- Em vários casos os campos da coluna 'name' estão preenchidos com palavras que não representam nomes e, nos casos de cachorro sem nome, está preenchida com a palavra 'None', em vez de o campo estar sisplesmente vazio;
- Está faltando o estágio de vida de muitos cachorros, além de que, nesses casos, os campos dessas variáveis foram preenchidos com a palavra 'None' ao invés de estarem simplesmente vazios;
- Um tweet em que a avaliação dada ao cachorro teve o denominador igual a zero;
- A tabela contém classificações que não são originais (retweets).

##### - tabela `image_predictions`
- As variáveis 'tweet_id' e 'img_num' são do tipo 'int', mas deveriam ser do tipo 'string';
- Os nomes de várias colunas não são sugestivos e não deveriam estar abreviados;
- Estão faltando dados de predição, considerando que o número de linhas dessa tabela é menor que o da tabela 'twitter_archive';
- Uso do caractere underline ('_') para separar os nomes compostos das raças de cachorro em vez de se utilizar espaço simples e as iniciais desses nomes não estão todas em caixa-alta.

##### - tabela `df_tweepy`
- Estão fantando alguns tweets que não foram encontrados na API Tweepy a partir dos IDs informados na tabela 'twitter_archive'.

### Problemas de estrutura:

- As colunas 'rating_numerator' e 'rating_denominator' da tabela 'twitter_archive' deveriam formar uma única coluna, chamada 'rating', com dados tipo float, representando a divisão do numerador pelo denominador;
- As colunas 'doggo', 'floofer', 'pupper' e 'puppo' da tabela 'twitter_archive' deveriam formar uma única coluna, chamada 'stage';
- As tabelas 'image_predictions' e 'df_tweepy' são tabelas independentes, mas deveriam ser somente partes da tabela 'twitter_archive'.

## Limpando os dados

In [691]:
#Fazendo uma cópia de cada tabela.
twitter_archive_clean = twitter_archive.copy()
image_predictions_clean = image_predictions.copy()
df_tweepy_clean = df_tweepy.copy()

### Dados ausentes

#### `Problema`: Está faltando o estágio de vida de muitos cachorros na tabela 'twitter_archive'

*Não há nada que possa ser feito nesse caso, pois, nesses casos, o texto do tweet simplesmente não informou qual era o estágio de vida*

#### `Problema`: Estão faltando dados de predição na tabela 'image_predictions'

*Também não há nada a fazer, pois não temos acesso à rede neural que gerou as predições*

#### `Problema`: Estão fantando alguns tweets na tabela 'df_tweepy' que não foram encontrados na API Tweepy

*Provavelmente o motivo de não conseguirmos localizar esses tweets na API Tweepy é porque eles devem ter sido apagados, de modo que não temos como resolver esse problema.*

### Tipo de variável inadequado

#### `Problema`: As variáveis 'timestamp' e 'retweeted_status_timestamp' são com tipo objeto (string), mas deveriam ser do tipo 'datatime'

##### Procedimento
*Converter essas duas variáveis do tipo 'string' para o tipo 'datatime'*

##### Código

In [692]:
twitter_archive_clean['timestamp'] = pd.to_datetime(twitter_archive_clean['timestamp'])
twitter_archive_clean['retweeted_status_timestamp'] = pd.to_datetime(twitter_archive_clean['retweeted_status_timestamp'])

##### Teste

In [693]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime

#### `Problema`: As variáveis que representam IDs, bem como a variável 'img_num', são do tipo 'int' ou 'float', mas deveriam ser do tipo 'string'

##### Procedimento
*Converter todas essas variáveis para o tipo 'string'. No caso das variáveis 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id'e 'retweeted_status_user_id', primeiramente colocar os valores não nulos no formato decimal em vez de notação científica e só depois fazer a conversão para 'string'. Depois que tudo tiver sido convertido, substituir cada ocorrência da palavra 'nan' pelo valor nulo.*

##### Código

In [694]:
#Coloca no formato decimal e então converte para 'string. Por fim, substitui a palavra 'nan' pelo valor nulo.
def edit_format(column, number_characters):
    for idx in twitter_archive_clean.index:
        if not math.isnan(twitter_archive_clean[column][idx]):
            twitter_archive_clean.loc[idx, column] = ('{:' + number_characters + '.0f}').format(
                                    twitter_archive_clean[column][idx])
    twitter_archive_clean[column] = twitter_archive_clean[column].astype(str)
    for idx, item in twitter_archive_clean[column].iteritems():
        if item == 'nan':
            twitter_archive_clean.loc[idx, column] = np.nan

In [695]:
twitter_archive_clean['tweet_id'] = twitter_archive_clean['tweet_id'].astype(str)
image_predictions_clean['tweet_id'] = image_predictions_clean['tweet_id'].astype(str)
image_predictions_clean['img_num'] = image_predictions_clean['img_num'].astype(str)
edit_format('in_reply_to_status_id', '18')
edit_format('in_reply_to_user_id', '10')
edit_format('retweeted_status_id', '18')
edit_format('retweeted_status_user_id', '10')

##### Teste

In [696]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null object
in_reply_to_status_id         78 non-null object
in_reply_to_user_id           78 non-null object
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null object
retweeted_status_user_id      181 non-null object
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime64[

In [697]:
image_predictions_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null object
jpg_url     2075 non-null object
img_num     2075 non-null object
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), object(6)
memory usage: 152.1+ KB


In [698]:
twitter_archive_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaT,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


### Variáveis com valores que não representam o seu conteúdo ou que deveriam estar com conteúdo vazio.

#### `Problema`: Em vários casos os campos da coluna 'name' estão preenchidos com palavras que não representam nomes e, nos casos de cachorro sem nome, está preenchida com a palavra 'None', em vez de o campo estar sisplesmente vazio.

##### Procedimento
*Substituir pelo valor nulo  todas as palavras que não representam nomes, inclusive a palavra (string) 'None'.*

##### Código

In [699]:
not_name = twitter_archive_clean['name'][twitter_archive_clean['name'].str.islower()]
not_name = not_name.append(pd.Series('None'), ignore_index=True)
for idx, is_true in twitter_archive_clean['name'].isin(not_name).iteritems():
    if is_true:
        twitter_archive_clean.loc[idx, "name"] = np.nan

##### Teste

In [700]:
twitter_archive_clean['name'].unique()

array(['Phineas', 'Tilly', 'Archie', 'Darla', 'Franklin', nan, 'Jax',
       'Zoey', 'Cassie', 'Koda', 'Bruno', 'Ted', 'Stuart', 'Oliver',
       'Jim', 'Zeke', 'Ralphus', 'Canela', 'Gerald', 'Jeffrey', 'Maya',
       'Mingus', 'Derek', 'Roscoe', 'Waffles', 'Jimbo', 'Maisey', 'Lilly',
       'Earl', 'Lola', 'Kevin', 'Yogi', 'Noah', 'Bella', 'Grizzwald',
       'Rusty', 'Gus', 'Stanley', 'Alfy', 'Koko', 'Rey', 'Gary', 'Elliot',
       'Louis', 'Jesse', 'Romeo', 'Bailey', 'Duddles', 'Jack', 'Emmy',
       'Steven', 'Beau', 'Snoopy', 'Shadow', 'Terrance', 'Aja', 'Penny',
       'Dante', 'Nelly', 'Ginger', 'Benedict', 'Venti', 'Goose', 'Nugget',
       'Cash', 'Coco', 'Jed', 'Sebastian', 'Walter', 'Sierra', 'Monkey',
       'Harry', 'Kody', 'Lassie', 'Rover', 'Napolean', 'Dawn', 'Boomer',
       'Cody', 'Rumble', 'Clifford', 'Dewey', 'Scout', 'Gizmo', 'Cooper',
       'Harold', 'Shikha', 'Jamesy', 'Lili', 'Sammy', 'Meatball',
       'Paisley', 'Albus', 'Neptune', 'Quinn', 'Belle', 'Zooey',

#### `Problema`:  Os valores das variáveis de estágio de vida foram preenchidos com a palavra (string) 'None' ao invés de estarem vazios.

##### Procedimento
*Substituir pelo valor nulo  todas as ocorrências da palavra 'None' nas colunas 'doggo', 'floofer', 'pupper' e 'puppo'*

##### Código

In [701]:
for column in ['doggo', 'floofer', 'pupper', 'puppo']:
    for idx, is_true in twitter_archive_clean[column].isin(['None']).iteritems():
        if is_true:
            twitter_archive_clean.loc[idx, column] = np.nan

##### Teste

In [702]:
#Checa o total de ocorrências da palavra 'None' nas colunas 'doggo', 'floofer', 'pupper' e 'puppo'
for column in ['doggo', 'floofer', 'pupper', 'puppo']:
    print(sum(twitter_archive_clean[column].isin(['None'])))

0
0
0
0


### Demais problemas de qualidade

#### `Problema`: Um tweet em que a avaliação dada ao cachorro teve o denominador igual a zero.

##### Procedimento
Apagar da tabela twitter_archive_clean a linha em que o valor da variável 'rating_denominator' é zero. Apagar também as linhas correspondentes nas demais tabelas (com o mesmo tweet_id), caso existam.

##### Código

In [703]:
tweet_id_deleted = twitter_archive_clean[twitter_archive_clean['rating_denominator'] == 0]['tweet_id'].values[0]
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['rating_denominator'] != 0]
image_predictions_clean = image_predictions_clean[image_predictions_clean['tweet_id'] != tweet_id_deleted]
df_tweepy_clean = df_tweepy_clean[df_tweepy_clean['tweet_id'] != tweet_id_deleted]

##### Teste

In [704]:
twitter_archive_clean[twitter_archive_clean['rating_denominator'] == 0].shape[0]

0

In [705]:
image_predictions_clean[image_predictions_clean['tweet_id'] == tweet_id_deleted].shape[0]

0

In [706]:
df_tweepy_clean[df_tweepy_clean['tweet_id'] == tweet_id_deleted].shape[0]

0

#### `Problema`:  Os nomes de várias colunas da tabela 'image_predictions' não são sugestivos e não deveriam estar abreviados

##### Procedimento
*Renomear as colunas com nomes mais adequados.*

##### Código

In [707]:
image_predictions_clean = image_predictions_clean.rename(index=str, columns={"img_num": "image_number", 
                                            "p1": "prediction1",
                                            "p2": "prediction2",
                                            "p3": "prediction3",
                                            "p1_conf": "prediction1_confidence",
                                            "p2_conf": "prediction2_confidence",
                                            "p3_conf": "prediction3_confidence",
                                            "p1_dog": "prediction1_is_dog",
                                            "p2_dog": "prediction2_is_dog",
                                            "p3_dog": "prediction3_is_dog"})

##### Teste

In [708]:
image_predictions_clean.head()

,tweet_id,jpg_url,image_number,prediction1,prediction1_confidence,prediction1_is_dog,prediction2,prediction2_confidence,prediction2_is_dog,prediction3,prediction3_confidence,prediction3_is_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


#### `Problema`:  Nomes compostos separados com underline ('_') e iniciais em caixa-baixa.

##### Procedimento

*Aplicar uma função que substitue o underline por espaço simples e coloca as iniciais em caixa-alta*

##### Código

In [709]:
def underscore_space(prediction, prediction_number):
    return prediction[prediction_number].replace('_', ' ').title()

In [710]:
image_predictions_clean['prediction1'] = image_predictions_clean.apply(underscore_space, args=('prediction1',), axis=1)
image_predictions_clean['prediction2'] = image_predictions_clean.apply(underscore_space, args=('prediction2',), axis=1)
image_predictions_clean['prediction3'] = image_predictions_clean.apply(underscore_space, args=('prediction3',), axis=1)

##### Teste

In [711]:
image_predictions_clean['prediction1'].value_counts()

Golden Retriever             150
Labrador Retriever           100
Pembroke                      89
Chihuahua                     83
Pug                           57
Chow                          44
Samoyed                       43
Toy Poodle                    39
Pomeranian                    38
Malamute                      30
Cocker Spaniel                30
French Bulldog                26
Miniature Pinscher            23
Chesapeake Bay Retriever      23
Seat Belt                     22
Staffordshire Bullterrier     20
German Shepherd               20
Siberian Husky                20
Web Site                      19
Cardigan                      19
Maltese Dog                   18
Shetland Sheepdog             18
Eskimo Dog                    18
Teddy                         18
Beagle                        18
Shih-Tzu                      17
Lakeland Terrier              17
Rottweiler                    17
Italian Greyhound             16
Kuvasz                        16
          

#### `Problema`:  A tabela 'twitter_archive_clean' contém classificações que não são originais (retweets).

##### Procedimento
*Deixar na tabela apenas as linhas em que o valor da variável 'retweeted_status_id' é nulo.*

##### Código

In [712]:
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean['retweeted_status_id'].isnull()]

##### Teste

In [713]:
twitter_archive_clean['retweeted_status_id'].value_counts()

Series([], Name: retweeted_status_id, dtype: int64)

In [714]:
twitter_archive_clean['retweeted_status_user_id'].value_counts()

Series([], Name: retweeted_status_user_id, dtype: int64)

In [715]:
twitter_archive_clean['retweeted_status_timestamp'].value_counts()

Series([], Name: retweeted_status_timestamp, dtype: int64)

#### `Novo problema`: Com as mudanças feitas na tabela 'twitter_archive_clean', as colunas 'retweeted_status_id', 'retweeted_status_user_id' e 'retweeted_status_timestamp' já não agregam mais nenhuma informação útil.

##### Procedimento
Apagar as colunas 'retweeted_status_id', 'retweeted_status_user_id' e 'retweeted_status_timestamp'.

##### Código

In [716]:
twitter_archive_clean = twitter_archive_clean.drop('retweeted_status_id', axis=1).drop( 'retweeted_status_user_id',
                                                                 axis=1).drop('retweeted_status_timestamp', axis=1)

##### Teste

In [717]:
twitter_archive_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN,NaN,NaN,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,NaN,NaN,NaN,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,NaN,NaN,NaN,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,NaN,NaN,NaN,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,NaN,NaN,NaN,NaN


### Problemas de estrutura

#### `Problema`:  As colunas 'rating_numerator' e 'rating_denominator' da tabela 'twitter_archive' deveriam formar uma única coluna

##### Procedimento
*Criar uma nova coluna chamada 'rating' para a tabela 'twitter_archive_clean'. Em seguida apagar as colunas 'rating_numerator' e 'rating_denominator'.*

##### Código

In [718]:
rating = []
for idx, row in twitter_archive_clean.iterrows():
    rating.append(twitter_archive_clean['rating_numerator'][idx] / twitter_archive_clean['rating_denominator'][idx])
twitter_archive_clean['rating'] = np.array(rating)
twitter_archive_clean = twitter_archive_clean.drop('rating_numerator', axis=1).drop('rating_denominator', axis=1)

##### Teste

In [719]:
twitter_archive_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,name,doggo,floofer,pupper,puppo,rating
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,Phineas,NaN,NaN,NaN,NaN,1.3
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,Tilly,NaN,NaN,NaN,NaN,1.3
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,Archie,NaN,NaN,NaN,NaN,1.2
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,Darla,NaN,NaN,NaN,NaN,1.3
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,Franklin,NaN,NaN,NaN,NaN,1.2


In [720]:
twitter_archive_clean['rating'].value_counts()

1.200000      504
1.000000      444
1.100000      431
1.300000      307
0.900000      156
0.800000       98
0.700000       53
1.400000       43
0.500000       37
0.600000       32
0.300000       19
0.400000       15
0.200000       11
0.100000        7
42.000000       2
0.000000        2
1.700000        1
1.250000        1
66.600000       1
1.500000        1
7.500000        1
3.428571        1
177.600000      1
0.818182        1
0.636364        1
18.200000       1
0.733333        1
2.700000        1
2.600000        1
Name: rating, dtype: int64

#### `Problema`:  As colunas 'doggo', 'floofer', 'pupper' e 'puppo' da tabela 'twitter_archive' deveriam formar uma única coluna, chamada 'stage'.

##### Procedimento
*Criar uma nova coluna chamada 'stage' que informe o estágio de vida do cachorro e depois apagar as colunas 'doggo', 'floofer', 'pupper' e 'puppo'*

##### Código

In [721]:
stage = [np.nan] * twitter_archive_clean.shape[0]
for idx, row in twitter_archive_clean.iterrows():
    for column in ['doggo', 'floofer', 'pupper', 'puppo']:
        if twitter_archive_clean[column][idx] == column:
            stage[idx] = column
twitter_archive_clean['stage'] = np.array(stage)
twitter_archive_clean = twitter_archive_clean.drop('doggo', axis=1).drop('floofer', axis=1).drop(
                                                            'pupper', axis=1).drop('puppo', axis=1)

##### Teste

In [722]:
twitter_archive_clean['stage'].unique()

array(['nan', 'doggo', 'puppo', 'pupper', 'floofer'], dtype=object)

In [723]:
twitter_archive_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,name,rating,stage
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,Phineas,1.3,nan
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,Tilly,1.3,nan
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,Archie,1.2,nan
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,Darla,1.3,nan
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,Franklin,1.2,nan


#### `Problema`: Os dataframes 'image_predictions' e 'df_tweepy' não deveriam ser tabelas independentes.

##### Procedimento
*Mesclar as tabelas 'image_predictions' e 'df_tweepy', unindo-as pelo 'tweet_id' em comum. Depois mesclar as 'twitter_archive_clean' e df_tweepy', também pelo 'tweet_id' em comum. Optamos por deixar na tabela 'twitter_archive_clean' apenas os tweets que estejam representados nas demais tabelas, de modo que os demais serão automaticamente excluídos quando mesclarmos as tabelas.*

##### Código

In [724]:
twitter_archive_clean = pd.merge(twitter_archive_clean, df_tweepy, on='tweet_id', how='right') 
twitter_archive_clean = pd.merge(twitter_archive_clean, image_predictions_clean, on='tweet_id', how='right') 

##### Teste

In [725]:
twitter_archive_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,name,rating,stage,...,image_number,prediction1,prediction1_confidence,prediction1_is_dog,prediction2,prediction2_confidence,prediction2_is_dog,prediction3,prediction3_confidence,prediction3_is_dog
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,Phineas,1.3,nan,...,1,Orange,0.097049,False,Bagel,0.085851,False,Banana,0.076110,False
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,Tilly,1.3,nan,...,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,Papillon,0.068957,True
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,Archie,1.2,nan,...,1,Chihuahua,0.716012,True,Malamute,0.078253,True,Kelpie,0.031379,True
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,Darla,1.3,nan,...,1,Paper Towel,0.170278,False,Labrador Retriever,0.168086,True,Spatula,0.040836,False
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,Franklin,1.2,nan,...,2,Basset,0.555712,True,English Springer,0.225770,True,German Short-Haired Pointer,0.175219,True


In [726]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2075 entries, 0 to 2074
Data columns (total 23 columns):
tweet_id                  2075 non-null object
in_reply_to_status_id     23 non-null object
in_reply_to_user_id       23 non-null object
timestamp                 1993 non-null datetime64[ns]
source                    1993 non-null object
text                      1993 non-null object
expanded_urls             1993 non-null object
name                      1349 non-null object
rating                    1993 non-null float64
stage                     1993 non-null object
retweet_count             2068 non-null float64
favorite_count            2068 non-null float64
jpg_url                   2075 non-null object
image_number              2075 non-null object
prediction1               2075 non-null object
prediction1_confidence    2075 non-null float64
prediction1_is_dog        2075 non-null bool
prediction2               2075 non-null object
prediction2_confidence    2075 non-null f

Gravaremos, então, a tabela twitter_archive_clean, com os dados devidamente limpos e estruturados, em um arquivo 'csv'.

In [727]:
twitter_archive_clean.to_csv('twitter_archive_master.csv', index=False)

## __EXPLORAÇÃO__

### Variáveis que interessam para a análise, classificadas por tipo:

##### Variáveis categóricas:
- tweet_id
- name
- stage
- prediction1
- prediction1_is_dog
- prediction2
- prediction2_is_dog
- prediction3
- prediction3_is_dog

##### Variáveis quantitativas:
- timestamp
- rating
- retweet_count
- favorite_count
- image_number
- prediction1_confidence
- prediction2_confidence
- prediction3_confidence